In [1]:
# USING PANDAS AND CREATING DATA FRAMES

# Need pandas to read data in csv files 
import pandas as pd

# Get files to load from correct location
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read files with panda -> Pandas DataFrame (df)
school_data_df = pd.read_csv(school_data_to_load)
#school_data_df
student_data_df = pd.read_csv(student_data_to_load)
#student_data_df.head()

# Fix incorrect names on the data frame
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

for wrong in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(wrong, "")

# Combine the data into a single dataset.
complete_df = pd.merge(student_data_df, school_data_df, on=["school_name", "school_name"])


/Users/gabrielatuma/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: The default value of regex will change from True to False in a future version.


In [2]:
# INFORMATION ABOUT THE DATA ANALYZED 

# Find the number of students = 39170
student_count = complete_df["Student ID"].count()
#student_count

# And schools = 15
school_count = len(complete_df["school_name"].unique())

# And the budget = 82932329558 > wrong, should count each school once
#total_budget = complete_df["budget"].sum()
#total_budget

# The budget = 24649428
total_budget = school_data_df["budget"].sum()
#total_budget

# Average reading score = 81.87784018381414
avg_reading_score = complete_df["reading_score"].mean()
#avg_reading_score

# Average math score = 78.98537145774827
avg_math_score = complete_df["math_score"].mean()
#avg_math_score

In [3]:
# PASSING PERCENTAGES

# 1 - Determine the passing grade. (Will olny give true or false response)
passing_reading = complete_df["reading_score"] >= 70
passing_math = complete_df["math_score"] >= 70

# 2 - Get the number of students who passed math and reading in separate DataFrames.
passing_reading = complete_df[complete_df["reading_score"] >= 70]
passing_math = complete_df[complete_df["math_score"] >= 70]

# 3 - Calculate the number of students who passed math and reading.
passing_reading_count = passing_reading["Student ID"].count()
passing_math_count = passing_math["Student ID"].count()

# 4 - Calculate the percentage of students who passed math and reading.
reading_percentage = (passing_reading_count / student_count) *100
math_percentage = (passing_math_count / student_count) *100

# 5 - Get the number of students who passed both math and reading in a DataFrame.
passing_both = complete_df[(complete_df["reading_score"] >= 70) & (complete_df["math_score"] >= 70)]

# 6 - Calculate the number of students who passed both math and reading.
passing_student_count = passing_both["Student ID"].count()

# 7 - Calculate the percentage of students who passed both math and reading.
both_percentage = (passing_student_count / student_count) *100

In [4]:
# DISTRICT SUMMARY

# Adding a list of values with keys to create a new DataFrame.
district_summary_df = pd.DataFrame(
          [{"Total Schools": school_count,
          "Total Students": student_count,
          "Total Budget": total_budget,
          "Average Math Score": avg_math_score,
          "Average Reading Score": avg_reading_score,
          "% Passing Math": math_percentage,
         "% Passing Reading": reading_percentage,
        "% Overall Passing": both_percentage}])

# FORMATING THE RESULTS
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:,.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:,.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:,.0f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:,.0f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:,.0f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",79.0,81.9,75,86,65


In [5]:
# CREATING A DATAFRAME USING SCHOOL NAME AS INDEX 

# Not a dataframe yet
per_school_types = school_data_df.set_index(["school_name"])["type"]

# Creating the dataframe so later we can add relevant information
#df = pd.DataFrame(per_school_types)

In [6]:
#CREATING SERIES WITH COMMON INDEX (SCHOOL NAME)

per_school_counts = school_data_df.set_index(["school_name"])["size"]

# OR per_school_counts = school_data_complete_df["school_name"].value_counts()
# per_school_counts

per_school_budget = school_data_df.set_index(["school_name"])["budget"]

# Calculate the per capita spending
per_school_capita = per_school_budget / per_school_counts

# Average scores
per_school_math = complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = complete_df.groupby(["school_name"]).mean()["reading_score"]


In [7]:
# PERCENTAGES PER SCHOOL

per_school_passing_math = complete_df[(complete_df["math_score"] >= 70)]
per_school_passing_reading = complete_df[(complete_df["reading_score"] >= 70)]

# Use groupby and mean to define the series for the averages 

per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

per_school_passing_math = per_school_passing_math / per_school_counts *100
per_school_passing_reading = per_school_passing_reading / per_school_counts *100

per_passing_both = complete_df[(complete_df["math_score"] >= 70) & (complete_df["reading_score"] >= 70)]
per_passing_both = per_passing_both.groupby(["school_name"]).count()["student_name"]
per_passing_both = per_passing_both / per_school_counts *100


In [8]:
# SUMMARY 

per_school_summary_df = pd.DataFrame({
             "School Type": per_school_types,
             "Total Students": per_school_counts,
             "Total School Budget": per_school_budget,
             "Per Student Budget": per_school_capita,
             "Average Math Score": per_school_math,
           "Average Reading Score": per_school_reading,
           "% Passing Math": per_school_passing_math,
           "% Passing Reading": per_school_passing_reading,
           "% Overall Passing": per_passing_both})
# FORMAT
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)
per_school_summary_df["Average Math Score"] = per_school_summary_df["Average Math Score"].map("{:,.1f}".format)
per_school_summary_df["Average Reading Score"] = per_school_summary_df["Average Reading Score"].map("{:,.1f}".format)
per_school_summary_df["% Passing Math"] = per_school_summary_df["% Passing Math"].map("{:,.0f}".format)
per_school_summary_df["% Passing Reading"] = per_school_summary_df["% Passing Reading"].map("{:,.0f}".format)
per_school_summary_df["% Overall Passing"] = per_school_summary_df["% Overall Passing"].map("{:,.0f}".format)

per_school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.0,81.0,67,82,55
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.1,84.0,94,97,91
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7,81.2,66,81,53
Ford High School,District,2739,"$1,763,916.00",$644.00,77.1,80.7,68,79,54
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.4,83.8,93,97,91
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.3,80.9,67,81,54
Holden High School,Charter,427,"$248,087.00",$581.00,83.8,83.8,93,96,89
Huang High School,District,2917,"$1,910,635.00",$655.00,76.6,81.2,66,81,54
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.1,81.0,66,81,54


In [9]:
# FINDING THE HIGHEST PERFORMING SCHOOLS

top_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=False)

top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.1,84.0,94,97,91
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.4,83.8,93,97,91
Pena High School,Charter,962,"$585,858.00",$609.00,83.8,84.0,95,96,91
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.4,83.8,93,97,91
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.3,84.0,94,97,91


In [10]:
# FINDING THE LOWEST PERFORMING SCHOOLS

low_schools = per_school_summary_df.sort_values(["% Overall Passing"])

low_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7,81.2,66,81,53
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.8,80.7,66,80,53
Ford High School,District,2739,"$1,763,916.00",$644.00,77.1,80.7,68,79,54
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.3,80.9,67,81,54
Huang High School,District,2917,"$1,910,635.00",$655.00,76.6,81.2,66,81,54


In [20]:
# ANALYSIS PER GRADE (NINTH-TWELFTH)

# DF for each grade
ninth_graders = complete_df[(complete_df["grade"] == "9th")]
tenth_graders = complete_df[(complete_df["grade"] == "10th")]
eleventh_graders = complete_df[(complete_df["grade"] == "11th")]
twelfth_graders = complete_df[(complete_df["grade"] == "12th")]

# Math scores
ninth_grade_math_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]
tenth_grade_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]
eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]
twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

# Reading scores
ninth_grade_reading_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_grade_reading_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_grade_reading_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_grade_reading_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]


school_name
Bailey High School       80.912451
Cabrera High School      84.287958
Figueroa High School     81.384863
Ford High School         80.662338
Griffin High School      84.013699
Hernandez High School    80.857143
Holden High School       84.698795
Huang High School        80.305983
Johnson High School      81.227564
Pena High School         84.591160
Rodriguez High School    80.376426
Shelton High School      82.781671
Thomas High School       83.831361
Wilson High School       84.317673
Wright High School       84.073171
Name: reading_score, dtype: float64

In [26]:
# SUMMARY PER GRADE - MATH

math_scores_by_grade = pd.DataFrame({
               "9th": ninth_grade_math_scores,
               "10th": tenth_grade_math_scores,
               "11th": eleventh_grade_math_scores,
               "12th": twelfth_grade_math_scores})
# FORMAT

math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:.1f}".format)
math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:.1f}".format)
math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:.1f}".format)
math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:.1f}".format)

# Remove the index name.
math_scores_by_grade.index.name = None

math_scores_by_grade.head()

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4


In [25]:
# SUMMARY PER GRADE - READING

reading_scores_by_grade = pd.DataFrame({
              "9th": ninth_grade_reading_scores,
              "10th": tenth_grade_reading_scores,
              "11th": eleventh_grade_reading_scores,
              "12th": twelfth_grade_reading_scores})
# FORMAT
reading_scores_by_grade["9th"] = reading_scores_by_grade["9th"].map("{:,.1f}".format)
reading_scores_by_grade["10th"] = reading_scores_by_grade["10th"].map("{:,.1f}".format)
reading_scores_by_grade["11th"] = reading_scores_by_grade["11th"].map("{:,.1f}".format)
reading_scores_by_grade["12th"] = reading_scores_by_grade["12th"].map("{:,.1f}".format)

# Remove the index name.
reading_scores_by_grade.index.name = None
    
reading_scores_by_grade.head()

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
